<a href="https://colab.research.google.com/github/prksh830/Healthcare/blob/main/dnn%2Blstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# gen_data.py
import numpy as np
import os
from tqdm import trange

def simulate_one_episode(N=40, area_size=100, E_init=1.0, trans_range=30,
                         time_slots=200, max_queue=5, traffic_high=4):
    # returns histories of shape (time_slots, N, feat_dim)
    # features: [energy, queue, degree, rand_link]
    pos = np.random.rand(N,2) * area_size
    energy = np.ones(N) * E_init
    queue = np.zeros(N, dtype=int)
    neighbors = [None]*N
    for i in range(N):
        neighbors[i] = [j for j in range(N) if j!=i and np.linalg.norm(pos[i]-pos[j])<=trans_range]
    feat_hist = np.zeros((time_slots, N, 4), dtype=float)
    action_hist = np.zeros((time_slots, N), dtype=int)
    for t in range(time_slots):
        # features
        for i in range(N):
            feat_hist[t,i,0] = energy[i]
            feat_hist[t,i,1] = queue[i]
            feat_hist[t,i,2] = len(neighbors[i])
            feat_hist[t,i,3] = np.random.rand()*1.0  # link-quality noise

        # expert optimizer: energy-aware greedy CH selection (top 20% energy-queue score)
        score = 0.7*energy - 0.3*queue
        idx_sorted = np.argsort(-score)
        num_CH = max(1, int(0.2*N))
        actions = np.zeros(N, dtype=int)
        actions[idx_sorted[:num_CH]] = 1
        action_hist[t,:] = actions

        # simulate traffic generation then send
        new_traffic = np.random.randint(0, traffic_high+1, size=N)
        queue += new_traffic
        packets_sent = np.minimum(queue, max_queue)
        # energy consumption
        E_tx = 0.01
        E_CH = 0.005
        energy = np.maximum(energy - E_tx*packets_sent - E_CH*actions, 0.0)
        # update queues (packets_sent removed)
        queue -= packets_sent

    return feat_hist, action_hist

def build_dataset(out_file='wsn_dataset.npz', episodes=200, seq_len=5, N=40):
    X_seqs = []
    y_next = []
    for ep in trange(episodes):
        feats, acts = simulate_one_episode(N=N, time_slots=200, traffic_high=4)
        T = feats.shape[0]
        for t in range(seq_len, T-1):
            # take seq_len history as input, target is action at t+1
            seq = feats[t-seq_len+1:t+1,:,:]  # shape (seq_len, N, feat_dim)
            # we'll flatten per-node sequences -> samples per node
            # for each node create sample
            for i in range(N):
                X_seqs.append(seq[:,i,:])    # shape (seq_len, feat_dim)
                y_next.append(acts[t+1,i])   # next time action for node i
    X_seqs = np.array(X_seqs)  # (samples, seq_len, feat_dim)
    y_next = np.array(y_next)  # (samples,)
    np.savez_compressed(out_file, X=X_seqs, y=y_next)
    print("Saved dataset:", out_file, "X shape:", X_seqs.shape, "y shape:", y_next.shape)

if __name__ == '__main__':
    os.makedirs('data', exist_ok=True)
    build_dataset(out_file='data/wsn_dataset.npz', episodes=300, seq_len=5, N=40)

100%|██████████| 300/300 [00:17<00:00, 17.25it/s]


Saved dataset: data/wsn_dataset.npz X shape: (2328000, 5, 4) y shape: (2328000,)


In [2]:
# train_model.py
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import tf2onnx
import onnx
import os

npz = np.load('data/wsn_dataset.npz')
X = npz['X']  # (samples, seq_len, feat_dim)
y = npz['y']  # (samples,)

# shuffle and split
perm = np.random.permutation(len(X))
X = X[perm]; y = y[perm]
split = int(0.8*len(X))
X_train, X_val = X[:split], X[split:]
y_train, y_val = y[:split], y[split:]

seq_len = X.shape[1]
feat_dim = X.shape[2]

# build model: per-node sequence in -> DNN+LSTM -> sigmoid
inp = layers.Input(shape=(seq_len, feat_dim), name='node_seq')
# LSTM branch
x = layers.LSTM(32, return_sequences=False)(inp)
# Flatten recent frame for DNN branch (spatial-like)
recent = layers.Lambda(lambda z: z[:,-1,:])(inp)   # latest timestep
d = layers.Dense(32, activation='relu')(recent)
d = layers.Dense(16, activation='relu')(d)
# Combine
c = layers.Concatenate()([x,d])
c = layers.Dense(32, activation='relu')(c)
out = layers.Dense(1, activation='sigmoid', name='prob')(c)

model = models.Model(inp, out)
model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
              loss='binary_crossentropy', metrics=['accuracy'])

model.summary()
# Train
history = model.fit(X_train, y_train, validation_data=(X_val,y_val),
                    epochs=10, batch_size=256)

# Save Keras and ONNX
os.makedirs('models', exist_ok=True)
model.save('models/dnn_lstm_model.h5')

# Export to ONNX
spec = (tf.TensorSpec((None, seq_len, feat_dim), tf.float32, name="node_seq"),)
output_path = "models/dnn_lstm_model.onnx"
model_proto, _ = tf2onnx.convert.from_keras(model, input_signature=spec, opset=13)
with open(output_path, "wb") as f:
    f.write(model_proto.SerializeToString())
print("Saved ONNX model:", output_path)

ModuleNotFoundError: No module named 'tf2onnx'

In [6]:
# train_model.py
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import tf2onnx
import onnx
import os

npz = np.load('data/wsn_dataset.npz')
X = npz['X']  # (samples, seq_len, feat_dim)
y = npz['y']  # (samples,)

# shuffle and split
perm = np.random.permutation(len(X))
X = X[perm]; y = y[perm]
split = int(0.8*len(X))
X_train, X_val = X[:split], X[split:]
y_train, y_val = y[:split], y[split:]

seq_len = X.shape[1]
feat_dim = X.shape[2]

# build model: per-node sequence in -> DNN+LSTM -> sigmoid
inp = layers.Input(shape=(seq_len, feat_dim), name='node_seq')
# LSTM branch
x = layers.LSTM(32, return_sequences=False)(inp)
# Flatten recent frame for DNN branch (spatial-like)
recent = layers.Lambda(lambda z: z[:,-1,:])(inp)   # latest timestep
d = layers.Dense(32, activation='relu')(recent)
d = layers.Dense(16, activation='relu')(d)
# Combine
c = layers.Concatenate()([x,d])
c = layers.Dense(32, activation='relu')(c)
out = layers.Dense(1, activation='sigmoid', name='prob')(c)

model = models.Model(inp, out)
model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
              loss='binary_crossentropy', metrics=['accuracy'])

model.summary()
# Train
history = model.fit(X_train, y_train, validation_data=(X_val,y_val),
                    epochs=10, batch_size=256)

# Save Keras and ONNX
os.makedirs('models', exist_ok=True)
model.save('models/dnn_lstm_model.h5')

# Export to ONNX
spec = (tf.TensorSpec((None, seq_len, feat_dim), tf.float32, name="node_seq"),)
output_path = "models/dnn_lstm_model.onnx"
model_proto, _ = tf2onnx.convert.from_keras(model, input_signature=spec, opset=13, custom_op_handlers={})
with open(output_path, "wb") as f:
    f.write(model_proto.SerializeToString())
print("Saved ONNX model:", output_path)

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ node_seq            │ (None, 5, 4)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_2 (Lambda)   │ (None, 4)         │          0 │ node_seq[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 32)        │        160 │ lambda_2[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_2 (LSTM)       │ (None, 32)        │      4,736 │ node_seq[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 16)        │        528 │ dense_6[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_2       │ (None, 48)        │          0 │ lstm_2[0][0],     │
│ (Concatenate)       │                   │            │ dense_7[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 32)        │      1,568 │ concatenate_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ prob (Dense)        │ (None, 1)         │         33 │ dense_8[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 7,025 (27.44 KB)

 Trainable params: 7,025 (27.44 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
7275/7275 ━━━━━━━━━━━━━━━━━━━━ 55s 7ms/step - accuracy: 0.7976 - loss: 0.5026 - val_accuracy: 0.8002 - val_loss: 0.4999
Epoch 2/10
7275/7275 ━━━━━━━━━━━━━━━━━━━━ 46s 6ms/step - accuracy: 0.7995 - loss: 0.5006 - val_accuracy: 0.8002 - val_loss: 0.4959
Epoch 3/10
7275/7275 ━━━━━━━━━━━━━━━━━━━━ 83s 7ms/step - accuracy: 0.8003 - loss: 0.4952 - val_accuracy: 0.8009 - val_loss: 0.4940
Epoch 4/10
7275/7275 ━━━━━━━━━━━━━━━━━━━━ 48s 7ms/step - accuracy: 0.8003 - loss: 0.4933 - val_accuracy: 0.8008 - val_loss: 0.4920
Epoch 5/10
7275/7275 ━━━━━━━━━━━━━━━━━━━━ 52s 7ms/step - accuracy: 0.8004 - loss: 0.4929 - val_accuracy: 0.8002 - val_loss: 0.4933
Epoch 6/10
7275/7275 ━━━━━━━━━━━━━━━━━━━━ 78s 7ms/step - accuracy: 0.8001 - loss: 0.4928 - val_accuracy: 0.8010 - val_loss: 0.4912
Epoch 7/10
7275/7275 ━━━━━━━━━━━━━━━━━━━━ 52s 7ms/step - accuracy: 0.8001 - loss: 0.4926 - val_accuracy: 0.8006 - val_loss: 0.4915
Epoch 8/10
7275/7275 ━━━━━━━━━━━━━━━━━━━━ 78s 7ms/step - accuracy: 0.8009 - loss: 0

ERROR:tf2onnx.tfonnx:rewriter <function rewrite_constant_fold at 0x7ca54000a660>: exception `np.cast` was removed in the NumPy 2.0 release. Use `np.asarray(arr, dtype=dtype)` instead.
ERROR:tf2onnx.tfonnx:rewriter <function rewrite_constant_fold at 0x7ca54000a660>: exception `np.cast` was removed in the NumPy 2.0 release. Use `np.asarray(arr, dtype=dtype)` instead.
ERROR:tf2onnx.tfonnx:rewriter <function rewrite_constant_fold at 0x7ca54000a660>: exception `np.cast` was removed in the NumPy 2.0 release. Use `np.asarray(arr, dtype=dtype)` instead.


Saved ONNX model: models/dnn_lstm_model.onnx


In [3]:
!pip install tf2onnx

INFO: pip is looking at multiple versions of onnx to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.8/455.8 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 100.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 15.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydf 0.13.0 requires protobuf<7.0.0,>=5.29.1, but you have protobuf 3.20.3 which is incompatible.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.3 which is incompatible.
tensorflow-metadata 1.17.2 requires protobuf>=4.25.2; python_version >= "3

In [7]:
# train_model.py
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import tf2onnx
import onnx
import os

npz = np.load('data/wsn_dataset.npz')
X = npz['X']  # (samples, seq_len, feat_dim)
y = npz['y']  # (samples,)

In [8]:
# shuffle and split
perm = np.random.permutation(len(X))
X = X[perm]; y = y[perm]
split = int(0.8*len(X))
X_train, X_val = X[:split], X[split:]
y_train, y_val = y[:split], y[split:]

seq_len = X.shape[1]
feat_dim = X.shape[2]

In [9]:
# build model: per-node sequence in -> DNN+LSTM -> sigmoid
inp = layers.Input(shape=(seq_len, feat_dim), name='node_seq')

In [10]:
# LSTM branch
x = layers.LSTM(32, return_sequences=False)(inp)

In [11]:
# Flatten recent frame for DNN branch (spatial-like)
recent = layers.Lambda(lambda z: z[:,-1,:])(inp)   # latest timestep
d = layers.Dense(32, activation='relu')(recent)
d = layers.Dense(16, activation='relu')(d)

In [12]:
# Combine
c = layers.Concatenate()([x,d])
c = layers.Dense(32, activation='relu')(c)
out = layers.Dense(1, activation='sigmoid', name='prob')(c)

model = models.Model(inp, out)
model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
              loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ node_seq            │ (None, 5, 4)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_3 (Lambda)   │ (None, 4)         │          0 │ node_seq[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 32)        │        160 │ lambda_3[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_3 (LSTM)       │ (None, 32)        │      4,736 │ node_seq[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_10 (Dense)    │ (None, 16)        │        528 │ dense_9[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_3       │ (None, 48)        │          0 │ lstm_3[0][0],     │
│ (Concatenate)       │                   │            │ dense_10[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_11 (Dense)    │ (None, 32)        │      1,568 │ concatenate_3[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ prob (Dense)        │ (None, 1)         │         33 │ dense_11[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 7,025 (27.44 KB)

 Trainable params: 7,025 (27.44 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
# Train
history = model.fit(X_train, y_train, validation_data=(X_val,y_val),
                    epochs=10, batch_size=256)

Epoch 1/10
7275/7275 ━━━━━━━━━━━━━━━━━━━━ 51s 7ms/step - accuracy: 0.7981 - loss: 0.5026 - val_accuracy: 0.8014 - val_loss: 0.4984
Epoch 2/10
7275/7275 ━━━━━━━━━━━━━━━━━━━━ 49s 7ms/step - accuracy: 0.7998 - loss: 0.5004 - val_accuracy: 0.8014 - val_loss: 0.4974
Epoch 3/10
7275/7275 ━━━━━━━━━━━━━━━━━━━━ 84s 7ms/step - accuracy: 0.7995 - loss: 0.5000 - val_accuracy: 0.8016 - val_loss: 0.4936
Epoch 4/10
7275/7275 ━━━━━━━━━━━━━━━━━━━━ 82s 7ms/step - accuracy: 0.7997 - loss: 0.4961 - val_accuracy: 0.8016 - val_loss: 0.4918
Epoch 5/10
7275/7275 ━━━━━━━━━━━━━━━━━━━━ 47s 7ms/step - accuracy: 0.8005 - loss: 0.4927 - val_accuracy: 0.8021 - val_loss: 0.4899
Epoch 6/10
7275/7275 ━━━━━━━━━━━━━━━━━━━━ 48s 7ms/step - accuracy: 0.7996 - loss: 0.4933 - val_accuracy: 0.8021 - val_loss: 0.4896
Epoch 7/10
7275/7275 ━━━━━━━━━━━━━━━━━━━━ 86s 7ms/step - accuracy: 0.7998 - loss: 0.4929 - val_accuracy: 0.8023 - val_loss: 0.4888
Epoch 8/10
7275/7275 ━━━━━━━━━━━━━━━━━━━━ 81s 7ms/step - accuracy: 0.8005 - loss: 0

In [15]:
# Save Keras and ONNX
os.makedirs('models', exist_ok=True)
model.save('models/dnn_lstm_model.h5')

In [16]:
# Export to ONNX
spec = (tf.TensorSpec((None, seq_len, feat_dim), tf.float32, name="node_seq"),)
output_path = "models/dnn_lstm_model.onnx"
model_proto, _ = tf2onnx.convert.from_keras(model, input_signature=spec, opset=13)
with open(output_path, "wb") as f:
    f.write(model_proto.SerializeToString())
print("Saved ONNX model:", output_path)

ERROR:tf2onnx.tfonnx:rewriter <function rewrite_constant_fold at 0x7ca54000a660>: exception `np.cast` was removed in the NumPy 2.0 release. Use `np.asarray(arr, dtype=dtype)` instead.
ERROR:tf2onnx.tfonnx:rewriter <function rewrite_constant_fold at 0x7ca54000a660>: exception `np.cast` was removed in the NumPy 2.0 release. Use `np.asarray(arr, dtype=dtype)` instead.
ERROR:tf2onnx.tfonnx:rewriter <function rewrite_constant_fold at 0x7ca54000a660>: exception `np.cast` was removed in the NumPy 2.0 release. Use `np.asarray(arr, dtype=dtype)` instead.


Saved ONNX model: models/dnn_lstm_model.onnx
